# Dataiku technical test

**Interviewee**: Vincent Barbosa Vaz

**Dataiku contacts**: Judith Müller, Josh Cooper and Adam Jelley

# Table of contents

1. [Introduction](#Introduction)
  1. [Instructions](#Instructions)
- [Load files, wait...?!](#Load-files,-wait...?!)
  1. [How to infer columns name ?](#)
- [Exploratory analysis](#Exploratory-analysis)
  1. [Checking for imbalanced data](#Checking-for-imbalanced-data)
  - [More analysis](#More-analysis)
  - [Checking for outliers](#Checking-for-outliers)
- [Cleaning and feature engineering](#Cleaning-and-feature-engineering)
  1. [Data cleaning](#Data-cleaning)
  - [Feature engineering](#Feature-engineering)
- [Models](#Models)
- [Pipeline](#Pipeline)
- [Conclusion](#Conclusion)
- [Sources](#Sources)
- [Appendix](#Appendix)

# Introduction

## Instructions

The following link lets you download an archive containing an “exercise” US Census dataset:  
http://thomasdata.s3.amazonaws.com/ds/us_census_full.zip

This US Census dataset contains detailed but anonymized information for approximately 300,000 people.

The archive contains 3 files:

1. A large training file (csv)
2. Another test file (csv)
3. A metadata file (txt) describing the columns of the two csv files (identical for both)

The goal of this exercise is to model the information contained in the last column (42nd), i.e., whether a person makes more or less than $50,000 per year, from the information contained in the other columns. The exercise here consists of modeling a binary variable.

Work with Python (or R) to carry out the following steps:

1. Load the train and test files.
2. Perform an exploratory analysis on the data and create some relevant visualisations.
3. Clean, preprocess, and engineer features in the training data, with the aim of building a data set that a model will perform well on.
4. Create a model using these features to predict whether a person earns more or less than $50,000 per year. Here, the idea is for you to test a few different models, and see whether there are any techniques you can apply to improve performance over your first results.
5. Choose the model that appears to have the highest performance based on a comparison between reality (the 42nd variable) and the model’s prediction. 
6. Apply your model to the test file and measure its real performance on it (same method as above).

The goal of this exercise is not to create the best or the purest model, but rather to describe the steps you took to accomplish it.  
Explain areas that may have been the most challenging for you.  
Find clear insights on the profiles of the people that make more than $50,000 / year. For example, which variables seem to be the most correlated with this phenomenon?  
Finally, you push your code on GitHub to share it with me, or send it via email.

Once again, the goal of this exercise is not to solve this problem, but rather to spend a few hours on it and to thoroughly explain your approach.

# Load files, wait...?!

**Import libraries**

Spoiler alert, here are the libraries being used in this notebook:

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import math
from importlib import reload

# Machine learning tools
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA

from scipy.stats.mstats import winsorize

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots

from warnings import filterwarnings
filterwarnings('ignore')

**Loading files**

The zip file contains three files:

|File                          |Information                    |
|:-----------------------------|:------------------------------|
|**census_income_learn.csv**   |Train dataset                  |
|**census_income_test.csv**    |Test dataset                   |
|**census_income_metadata.txt**|Metadata: feature's information|

A first look on **training and test dataset** shows that they **don't have header** (columns names). We will need to get this information from metadata file in order to do feature engineering.

I have created a Python module to read through metadata file (the code is not very interesting from a ML point of view and it adds some scrolling, that's why it is on a module).

In [ ]:
import read_metadata
reload(read_metadata);

In [ ]:
meta_top = read_metadata.top()
meta_top.head(3)

In [ ]:
meta_middle = read_metadata.middle()
meta_middle.head(3)

In [ ]:
meta_bottom = read_metadata.bottom()
meta_bottom.head(3)

In [ ]:
df_train = pd.read_csv("data/census_income_learn.csv", header=None, na_values='?', skipinitialspace=True)
names = pd.Series(list(df_train.columns), name='feature').astype('category')

In [ ]:
merged = pd.concat([names, meta_top.iloc[:,0], meta_middle.iloc[:,0], meta_bottom.iloc[:,0]], axis=1)
merged.columns = ['df_train', 'meta_top', 'meta_middle', 'meta_bottom']
merged.tail(6)

**Observations**

- Training dataset contains 42 features unnamed.
- Features in metadata file are not described consistenly: length of decribed feature vary.
- It is not straightforward to infer df_train features names from metadata file.
- Features descriptions at the end of metadata file seems to be a better fit (matching length when adding income feature).

Merge of features information:

In [ ]:
pd.merge(meta_top, meta_middle).merge(meta_bottom).head(5)

**Note**: Feature names vary in file resulting in missing features during merge. An in-depth study would allow us to correctly match all the variables.

### How to infer columns name ?

**Simple idea**

As length of feature description at the end of metadata file (bottom file) closely match with the one of training dataset columns we can assume the order is correct and infer.

**More complex idea, a path to autoML**

To find the right columns names for our datasets one can use information on features in metadata file. We will only focus on the information from the last descriptions of features (bottom file).

The cell below contains unique values for **second column** from **training dataset** we want to find feature name:

In [ ]:
list(df_train.iloc[:, 1].unique())

The cell below contains unique values for feature **class of worker** from **metadata file**:

In [ ]:
meta_bottom.iloc[1, 1]

**Observations**

- Values are similar
- There is a high probability for 2nd column of training dataset to be feature class of worker


How to measure "distance" between metadata file features and every training dataset column?

Jaccard similarity is a distance metric that can be used to measure distance between two lists.

In [ ]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def compute_similarity(index):
    similarities = []
    for i in range(len(df_train.columns)):
        similarities.append([round(jaccard_similarity(list(df_train.iloc[:, i].unique()),
                                                      meta_bottom.iloc[index, 1]),2),
                             meta_bottom.iloc[index, 0],
                             df_train.columns.values[i]
                            ])
        similarities.sort(reverse=True)

    # Select most miningful features
    sim = [x[0] for x in similarities]
    ind = [i+1 for i,x in enumerate(sim[1:]) if(sim[0]-x<sim[0]-sim[1]+0.05
                                                and sim[0]-sim[1] <0.1
                                                and sim[0] >0)]
    if(sim[0]>0): ind.insert(0, 0)

    #print("3-closest features:")
    #print(similarities[:5])
    #print()
    if len(ind)>0:
        for i in ind:
            print("\"{}\" match \"{}\" ({}%)".format(similarities[i][1],
                                             similarities[i][2],
                                             similarities[i][0]*100))
        print()
    
for i in range(len(meta_bottom)):
    compute_similarity(i)


The code above measure distances between metadata file and training dataset columns, returning the bests matches.

**Conclusion**

We introduced Jaccard similarity to infer information on missing data (columns names). One can create such data to automatically deduce the type of data (autoML for feature engineering).

Ultimately we will use the insights given by Jaccard similarity to confirm our pairing.

**Pairing method**:

1. parse metadata file (last feature descriptions);
2. inpute missing column names from parsed data (same order);
3. manual validation step with Jaccard similarity;
4. construction of names.csv file containing the header for training and test dataset;
5. read training and test files setting names attribute with content from names.csv

**Load files (with header)**

Now we are ready to load our datasets with the corresponding column names.

In [ ]:
names = pd.read_csv("data/names.csv", sep=',').columns.tolist()

df_train = pd.read_csv("data/census_income_learn.csv", header=None, na_values='?',
                       names=names, skipinitialspace=True)

df_test = pd.read_csv("data/census_income_test.csv", header=None, na_values='?',
                      names=names, skipinitialspace=True)

## Exploratory analysis

Performing an exploratory analysis on the data and create some relevant visualizations.

In [ ]:
df_train.head(3)

In [ ]:
df_train.info()

**Note**

**Meaning of value *'Not in Universe'*:** According to the IPUMS website (https://cps.ipums.org/cps-action/faq), indicates that the census question was irrelevant to the households or persons to whom the question was asked. 

### Checking for imbalanced data

Repartition of income:

In [ ]:
income_rep = df_train['income'].value_counts(normalize=True).mul(100).round(2).reset_index()
income_rep

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=[income_rep['income'][0]],
    name=income_rep['index'][0],
    orientation='h'
))

fig.add_trace(go.Bar(
    x=[income_rep['income'][1]],
    name=income_rep['index'][1],
    orientation='h'
))

fig.update_layout(barmode='stack', title = "Distribution of low and high income")
fig.show()

**Observation**: *'income'* has two unique values *'50 000+'* and *'- 50 000'* with a ratio 1:9

Let's analyze the distribution of income for *'race'* feature:

In [ ]:
crosstab = pd.crosstab(df_train['race'], df_train['income'])
crosstab = crosstab.sort_values(crosstab.columns[0],axis=0,ascending=False)
crosstab

In [ ]:
fig = go.Figure()

for i in range(len(crosstab.columns)):
    fig.add_trace(go.Bar(x=crosstab.index, y=crosstab.values[:,i], name=crosstab.columns[i]))
    
fig.update_layout(
    title= "# of Income vs. Race",
    xaxis_title = "Race",
    yaxis_title = "# of Income",
    font=dict(size=12)
)

fig.show()

**Observation**: Ethnicity is imbalanced, white people are overrepresented

Let's create a function for further plotting:

In [ ]:
def countplot(data, x, hue, values=None, aggfunc=None):
    """
    Implementation of sns.countplot for Plotly.
    """
    
    crosstab = pd.crosstab(df_train[x], df_train[hue], values=values, aggfunc=aggfunc)
    crosstab = crosstab.sort_values(crosstab.columns[0],axis=0,ascending=False)

    fig = go.Figure()

    for i in range(len(crosstab.columns)):
        fig.add_trace(go.Bar(x=crosstab.index, y=crosstab.values[:,i], name=crosstab.columns[i]))
    
    fig.update_layout(
        title= hue.capitalize() + " vs. " + x.capitalize() ,
        xaxis_title = x,
        yaxis_title = hue,
        font=dict(size=12)
    )

    return fig

In [ ]:
fig = countplot(data=df_train, x='sex', hue='income')
fig.update_layout(
    title= '# of Income vs. Sex',
    xaxis_title = 'Sex',
    yaxis_title = '# of Income',
    font=dict(size=12),
    height=500
)
fig.show()

**Observations**

- Female and male are balanced (total number of income equal)
- Nevertheless, fewer men make less than $50 000 comparing to women

### More analysis

In [ ]:
fig = countplot(data=df_train, x='sex', hue='income', values=df_train['age'], aggfunc=np.mean)
fig.update_layout(
    title= 'Average age vs. sex /income',
    xaxis_title = 'sex',
    yaxis_title = 'average age',
    font=dict(size=12),
    height=400
)
fig.show()

**Observations**

- Revenue for men and women is unequal
- The average age for women making less than $50 000 is 35 while it is 32 for men
- Men start earning more money earlier than women do

**Tree map and geographical plots**

Let's focus on the "state of previous residence" feature.

We would like to determine which state most of the high-income earners come from. State of previous residence may be involved in tax income (related to a higher or lower income).

In [ ]:
states_count = pd.crosstab(df_train["state of previous residence"], df_train["income"])
states_count = states_count.sort_values(crosstab.columns[0],axis=0,ascending=False)

states_count = states_count.reset_index()
states_count.rename(columns={'state of previous residence':'state', 
                             '- 50000.':'under50000', 
                             '50000+.':'above50000', }, inplace=True)
states_count["pct"] = states_count.apply (lambda row: row.above50000/(row.above50000 + row.under50000)*100, axis=1)


states_count = states_count[~states_count.state.str.contains("Not in universe")]
states_count.head(3)

In [ ]:
is_US = ["not US" if (x == 'Abroad') else "US" for x in states_count['state']] 

tree = px.treemap(states_count, 
                path = [is_US,"state"],
                values="pct",
                color= "state")


tree.show()

In the previous treemap, we group the US states in a same "box". Connecticut, New Jersey, Alaska, Massachusetts and Columbia are home to the high income earners (previous residence state). But is there any geographical correlation ?

In [ ]:
us_states = pd.read_csv("data/us_states.csv")
us_states.rename(columns={'State':'state'}, inplace=True)


merged = states_count.merge(us_states, on="state")

fig = px.choropleth(merged,
                    locations="Code",
                    color="pct",
                    hover_name="state",
                    locationmode = 'USA-states')
fig.update_layout(
    title_text = 'Pourcentage of high income earners - state of their previous residence',
    geo_scope='usa',
)

fig.show()

By mapping the data in a choropleth map, we can see that most of those states are neighbors, and in the East coast of the United States. For a deeper analysis, we can study the migration code-change features. 

**Plotting correlation heatmap**

In [ ]:
corrs = df_train.corr()
figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)
figure

**Observations**

- As we see above *'weeks worked in year'* and *'detailed industry recode'* **have high correlation** (0.75). *'veterans benefits'* and *'age'* (0.67), *'detailed occupation recode'* and *'weeks worked in year'* (0.66), *'num persons worked for employer'* and *'detailed industry recode'* (0.64) too.

- *'detailed industry recode'* and *'detailed occupation recode'* have **medium correlation** between *'veterans benefits'*.

- The reminder columns have **low correlation**.

**Scatter plot**

In [ ]:
fig = px.scatter_matrix(
    df_train,
    dimensions=["weeks worked in year",
                "detailed occupation recode",
                "capital gains",
                "num persons worked for employer",
                "detailed industry recode",
                "veterans benefits"
               ],
    color="income")

fig.update_layout(
    title="Scatter plot",
    font=dict(
        size=9
    )
)
fig.show()

### Checking for outliers

To identify outliers we will use boxplots:

In [ ]:
fig = go.Figure(make_subplots(rows=2, cols=2))
fig.add_trace(go.Box(y=df_train['age'], name="age"), row=1, col=1)
fig.add_trace(go.Box(y=df_train['wage per hour'], name="wage per hour"), row=1, col=2)
fig.add_trace(go.Box(y=df_train['capital gains'], name="capital gains"), row=2, col=1)
fig.add_trace(go.Box(y=df_train['capital losses'], name="capital losses"), row=2, col=2)
fig.update_layout(title='Boxplots')
fig.show()

**Observations**

- Feature **age** does not have outliers, we will not apply outliers ridding techniques
- Other features plots does

In-depth analysis:

In [ ]:
fig = px.box(df_train, x="race", y="age", title="Box plots", notched=True, height=400)
fig.show()

In-depth analysis:

In [ ]:
fig = px.box(df_train, x="race", y="age", color="sex", height=400)
fig.show()

In [ ]:
fig = px.histogram(df_train, x="age", color="sex", y="income", nbins=50, height=400)
fig.show()

In [ ]:
fig = px.histogram(df_train, x="age", color="income", y="sex", nbins=50)
fig.show()

## Cleaning and feature engineering

### Data cleaning

#### Checking for missing data

Numerical features:

In [ ]:
df_train.select_dtypes(include=['number']).isna().sum(axis = 0)

Categorical features:

In [ ]:
df_train.select_dtypes(include=['object']).isna().sum(axis = 0)

**Observations**

- There is no missing data for numerical features
- Nearly 100 000 NaN values for *migration code* features

We will transform and fill missing values for categorical variables.

#### Removing outliers

Based on our previous analysis we will remove outliers.

Apply winsorization:

In [ ]:
df_train["capital gains"] = winsorize(df_train["capital gains"],(0,0.035))
df_train["capital losses"] = winsorize(df_train["capital losses"],(0,0.019))
df_train["wage per hour"] = winsorize(df_train["wage per hour"],(0,0.057))

Re-plot (with winsorization):

In [ ]:
fig = go.Figure(make_subplots(rows=2, cols=2))
fig.add_trace(go.Box(y=df_train['age'], name="age"), row=1, col=1)
fig.add_trace(go.Box(y=df_train['wage per hour'], name="wage per hour"), row=1, col=2)
fig.add_trace(go.Box(y=df_train['capital gains'], name="capital gains"), row=2, col=1)
fig.add_trace(go.Box(y=df_train['capital losses'], name="capital losses"), row=2, col=2)
fig.update_layout(title='Boxplots')
fig.show()

### Feature engineering

#### PCA: numerical features

In [ ]:
df_train_num = df_train._get_numeric_data()

df_stand = StandardScaler().fit_transform(df_train_num)
pca = PCA(n_components=0.9, whiten=True)
df_pca = pca.fit_transform(df_stand)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
print('Original number of features', df_stand.shape[1])
print('Reduced number of features', df_pca.shape[1])

In [ ]:
fig = px.line(y=pca.explained_variance_ratio_,
              title='PCA - Total variance explained: {0:.2f}'.format(pca.explained_variance_ratio_.sum()),
              height=500)
fig.show()

#### PCA: all features

In [ ]:
df_dummies = pd.get_dummies(df_train.drop(columns=['income']).dropna())

df_stand = StandardScaler().fit_transform(df_dummies)
pca = PCA(n_components=0.9, whiten=True)
df_pca = pca.fit_transform(df_stand)

In [ ]:
print('Original number of features', df_stand.shape[1])
print('Reduced number of features', df_pca.shape[1])

In [ ]:
fig = px.line(y=pca.explained_variance_ratio_,
              title='PCA - Total variance explained: {0:.2f}'.format(pca.explained_variance_ratio_.sum()),
              height=500)
fig.show()

# Models

In [ ]:
X = pd.get_dummies(df_train.dropna().drop(columns=['income']))
y = df_train.dropna()['income']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
logreg = LogisticRegression(max_iter = 100)
model_1 = logreg.fit(X_train, y_train)

pred_1 = model_1.predict(X_test)

print(classification_report(y_test, pred_1, target_names = ["+50 000", "-50 000"]))

In [ ]:
randf = RandomForestClassifier(n_estimators = 100)
model_2 = randf.fit(X_train, y_train)

pred_2 = model_2.predict(X_test)

print(classification_report(y_test, pred_2, target_names = ["+50 000", "-50 000"]))

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.5, max_depth=1)
model_3 = gbc.fit(X_train, y_train)

pred_3 = model_3.predict(X_test)

print(classification_report(y_test, pred_3, target_names = ["+50 000", "-50 000"]))

# Pipeline

In [ ]:
names = pd.read_csv("data/names.csv", sep=',').columns.tolist()

df_train = pd.read_csv("data/census_income_learn.csv", header=None, na_values='?',
                       names=names, skipinitialspace=True)

df_test = pd.read_csv("data/census_income_test.csv", header=None, na_values='?',
                      names=names, skipinitialspace=True)

In [ ]:
X = df_train.drop(columns=['income'])
y = df_train['income']

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['number']).columns

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('drop_columns', 'drop', ['instance weight']),
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

In [ ]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('RandomForestClassifier', RandomForestClassifier())
])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
model_pipeline.fit(X_train, y_train)

In [ ]:
y_pred_train = model_pipeline.predict(X_train)

In [ ]:
accuracy_score(y_train, y_pred_train)

In [ ]:
y_pred_val = model_pipeline.predict(X_val)

In [ ]:
accuracy_score(y_val, y_pred_val)

---

## Prediction for test file

In [ ]:
X_test = df_test.drop(columns=['income'])
y_test = df_test['income']

### Data check

Let's have a look on test dataset even if **we will not use this information to tune our model**.

In [ ]:
df_test.head(1)

**Note**: Great, columns in test dataset seems to match with header from train dataset. Luckily!

In [ ]:
df_test.select_dtypes(include=['number']).isna().sum(axis = 0)

In [ ]:
df_test.select_dtypes(include=['object']).isna().sum(axis = 0)

**Note**: Same feature repartition for NaNs. Numerical features don't have missing values (our model don't support numerical filling NaNs yet). Luckily!

### Prediction

Fitting model with entire train dataset.

In [ ]:
model_pipeline.fit(X, y)

In [ ]:
y_pred_train = model_pipeline.predict(X)

In [ ]:
accuracy_score(y, y_pred_train)

In [ ]:
y_pred_test = model_pipeline.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_test)

## Conclusion

**Columns names, metadata file**

It is the first time I encounter datasets with header missing. Usually, metadata files come with training and test sets with further information for context understanding and feature engineering. It gave me the idea to build a matching feature algorithm that could be used for autoML purposes.

**Results**

We got 0.95 accuracy on test data, impressive! (Did I do something wrong ?)

**What to do next?**

Compare other ML models, use cross validation etc. Validate the model for production purposes (performances).

## Sources

- https://www.kdnuggets.com/2018/08/make-machine-learning-models-robust-outliers.html
- https://stackoverflow.com/questions/14720324/compute-the-similarity-between-two-lists

## Appendix

Cosine similarity implementation to compute distance between two lists. Alternative to Jaccard distance.

Alternative to [Crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) is [Pivot Table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html). For this purpose Crosstab has a more succinct syntax. These two lines are equivalent:

Plotly alternative to plot histogram categorical variables:

Plotly alternative to plot histogram:

Plotly static: